# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [24]:
#Initial imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [25]:
#Load the .env variable with the Mapbox API key and set the mapbox access token
load_dotenv("MY_KEYS.env")
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [26]:
#Import the necessary CSVs that were output from the analysis code to new dataframes here.
file_path_1 = Path("Output_csv_data_files/combined_location_price_df.csv")
combined_location_price_df = pd.read_csv(file_path_1)

file_path_2 = Path("Output_csv_data_files/sfo_expensive_neighborhoods.csv")
sfo_expensive_neighborhoods = pd.read_csv(file_path_2)

file_path_3 = Path("Output_csv_data_files/sfo_expensive_neighborhoods_by_year.csv")
sfo_expensive_neighborhoods_by_year = pd.read_csv(file_path_3)

file_path_4 = Path("Raw_Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path_4, index_col="year")

file_path_5 = Path("Output_csv_data_files/sfo_mean_price_rent.csv")
sfo_mean_price_rent = pd.read_csv(file_path_5, index_col="year")

file_path_6 = Path("Output_csv_data_files/sfo_price_by_neighborhood.csv")
sfo_price_by_neighborhood = pd.read_csv(file_path_6, index_col="year")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [57]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    sfo_mean_housing_units = sfo_data.groupby(sfo_data.index)["housing_units"].mean()
    minimum_mean_units = sfo_mean_housing_units.min()
    maximum_mean_units = sfo_mean_housing_units.max()
    std_mean_units = sfo_mean_housing_units.std()
    
    sfo_mean_housing_units_plot = sfo_mean_housing_units.hvplot.bar(
        ylim=(minimum_mean_units - std_mean_units,maximum_mean_units + std_mean_units), 
        xlabel="Year", 
        ylabel="Average Housing Units", 
        title="Average Housing Units in San Francisco 2010-2016", 
        frame_width=750,
        frame_height=250
    )
      
    return sfo_mean_housing_units_plot

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_mean_rent_plot = sfo_mean_price_rent["avg_gross_rent"].hvplot(
        xlabel="Year", 
        ylabel="Average Gross Rent", 
        title="Average Gross Rent in San Francisco 2010-2016",
        color="indigo",
        frame_width=750,
        frame_height=250
    )
    
    return sfo_mean_rent_plot

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_mean_price_plot = sfo_mean_price_rent["avg_sale_price_sqr_foot"].hvplot(
        xlabel="Year", 
        ylabel="Average Sale Price per SqFt", 
        title="Average Sale Price per SqFt in San Francisco 2010-2016",
        color="seagreen",
        frame_width=750,
        frame_height=250
    )
    
    return sfo_mean_price_plot

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    sfo_average_sale_price_by_neighborhood_plot = sfo_price_by_neighborhood.hvplot(
        y="avg_sale_price_sqr_foot", 
        x="year", 
        groupby="neighborhood", 
        ylabel="Average Sale Price per SqFt", 
        xlabel="Year")
    
    return sfo_average_sale_price_by_neighborhood_plot

def average_gross_rent_by_neighborhood():
    """Average Gross Rent by Neighborhood."""
    
    sfo_avg_gross_rent_by_neighborhood_plot = sfo_price_by_neighborhood.hvplot(
        y="avg_gross_rent", 
        x="year", 
        groupby="neighborhood", 
        ylabel="Average Gross Rent", 
        xlabel="Year")
    
    return sfo_avg_gross_rent_by_neighborhood_plot
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    sfo_expensive_neighborhoods_plot = sfo_expensive_neighborhoods.hvplot.bar(
        title="Top 10 Most Expensive Neighborhoods in San Francisco", 
        y="avg_sale_price_sqr_foot",
        x="neighborhood", 
        ylabel="Average Sale Price per SqFt", 
        xlabel="Neighborhood",
        rot=90,
        frame_width=750,
        frame_height=250,
        color="orangered")
    
    return sfo_expensive_neighborhoods_plot

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
       
    comparison_rent_price_plot = sfo_price_by_neighborhood.hvplot.bar(
        title="Average Purchase Price per SqFt vs. Average Gross Rent by Year", 
        y=["avg_gross_rent","avg_sale_price_sqr_foot"],
        groupby="neighborhood",
        ylabel="Average Gross Rent/Average Sale Price per SqFt",
        xlabel="Year",
        rot=90,
        frame_width=750,
        frame_height=250,
        cmap=["darkblue","darkred"])
    
    return comparison_rent_price_plot
    
def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    parallel_coordinates_plot = px.parallel_coordinates(
        sfo_expensive_neighborhoods, 
        color="avg_sale_price_sqr_foot",
        labels={
            "avg_sale_price_sqr_foot": "Avg. Sale Price per SqFt",
            "avg_housing_units": "Avg. Housing Units",
            "avg_gross_rent":"Avg. Gross Rent"
        },
        title="Parallel Coordinates Analysis of Most Expensive Neighborhoods in San Francisco Per Year",
        width=1000,
        height=750)

    parallel_coordinates_plot_pane = pn.pane.Plotly(parallel_coordinates_plot)
    
    return parallel_coordinates_plot_pane

def parallel_categories():
    """Parallel Categories Plot."""
    
    parallel_categories_plot = px.parallel_categories(
        sfo_expensive_neighborhoods,
        dimensions=["neighborhood", "avg_sale_price_sqr_foot", "avg_housing_units","avg_gross_rent"],
        color="avg_sale_price_sqr_foot",
        color_continuous_scale="reds",
        labels={
            "neighborhood": "Neighborhood",
            "avg_sale_price_sqr_foot": "Avg. Sale Price per SqFt",
            "avg_housing_units": "Avg. Housing Units",
            "avg_gross_rent":"Avg. Gross Rent"
        },
        title="Parallel Categories Analysis of Most Expensive Neighborhoods in San Francisco Per Year",
        width=1000,
        height=750)
    
    parallel_categories_plot_pane = pn.pane.Plotly(parallel_categories_plot)
    
    return parallel_categories_plot_pane

def neighborhood_map():
    """Neighborhood Map."""
    
    sfo_price_map_plot = px.scatter_mapbox(
        combined_location_price_df,
        lat="Lat",
        lon="Lon",
        size="avg_sale_price_sqr_foot",
        color="avg_gross_rent",
        title="Average Price per SqFt (by size) and Average Gross Rent (by color) per Neighborhood",
        hover_data=["Neighborhood","Lat","Lon","avg_sale_price_sqr_foot","avg_gross_rent"],
        color_continuous_scale="reds",
        zoom=11,
        width=1000,
        height=750)
    
    sfo_price_map_plot_pane = pn.pane.Plotly(sfo_price_map_plot)
    sfo_price_map_plot_pane._updates=True
    return sfo_price_map_plot_pane

def sunburst():
    """Sunburst Plot."""
    
    sunburst_plot = px.sunburst(
        sfo_expensive_neighborhoods_by_year, 
        path=["year","neighborhood"],
        values="sale_price_sqr_foot",
        color="gross_rent",
        labels={"gross_rent":"Gross Rent"},
        width=1000,
        height=750,
        title="Cost Analysis of Most Expensive Neighborhoods in San Francisco Per Year",
        color_continuous_scale="reds")
    
    sunburst_plot_pane = pn.pane.Plotly(sunburst_plot)
    
    return sunburst_plot_pane

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [99]:
#Create the panels for the dashboard
welcome_column = pn.Column(
    "###This dashboard presents a visual analysis of historical prices of housing units, sale price per square foot, and gross rent in San Francisco, California from 2010-2016. You can navigate through the tabs above to explore more details about the evolution of the real estate market on the The Golden City across the years.", 
    neighborhood_map())

yearly_market_analysis_row = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())

parallel_plots_column = pn.Column(parallel_coordinates(),parallel_categories())

sunburst_column = pn.Column(sunburst())

neighborhood_column_1 = pn.Column(average_price_by_neighborhood(), average_gross_rent_by_neighborhood())
neighborhood_column_2 = pn.Column(top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())
neighborhood_analysis_grid = pn.GridSpec(width=2000, height=1000)
neighborhood_analysis_grid[:,0] = neighborhood_column_1
neighborhood_analysis_grid[:,1] = neighborhood_column_2

#Create a tab layout for the dashboard
SFO_real_estate_dashboard = pn.Tabs(
    ("Welcome", welcome_column),
    ("Yearly Market Analysis", yearly_market_analysis_row),
    ("Neighborhood Analysis", neighborhood_analysis_grid),
    ("Parallel Plots Analysis", parallel_plots_column),
    ("Sunburst Plot Analysis", sunburst_column)
)

## Serve the Panel Dashboard

In [100]:
#Serve the dashboard
SFO_real_estate_dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Plotly(Figure)
    [1] Row
        [0] HoloViews(Bars)
        [1] HoloViews(Curve)
        [2] HoloViews(Curve)
    [2] GridSpec(height=1000, ncols=2, nrows=1, sizing_mode='fixed', width=2000)
        [0] Column(height=1000, width=1000)
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
        [1] Column(height=1000, width=1000)
            [0] HoloViews(Bars)
            [1] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
    [3] Column
        [0] Plotly(Figure, visible=False)
        [1] Plotly(Figure, visible=False)
    [4] Column
        [0] Plotly(Figure, visible=False)

# Debugging

This is a section for debugging the code only. Uncomment the following code to test each plot.

In [92]:
#housing_units_per_year()

In [93]:
#average_gross_rent()

In [94]:
#average_sales_price()

In [95]:
#average_price_by_neighborhood()

In [68]:
#average_gross_rent_by_neighborhood()

In [69]:
#top_most_expensive_neighborhoods()

In [96]:
#most_expensive_neighborhoods_rent_sales()

In [97]:
#neighborhood_map()

In [98]:
#parallel_categories()

In [99]:
#parallel_coordinates()

In [52]:
#sunburst()